In [6]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed


In [7]:
using JuMP
using MathProgBase
using GLPKMathProgInterface

In [32]:
include("../utils/util.jl")
include("../utils/problem.jl")

In [33]:
small_nnet = read_nnet("small_nnet.txt")

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5]), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5]), Layer([3.0 3.0], [3.5])])

* Input and output constraints in helper file
* call function encode - aim is to transform a network, input constraints, and output constraints into a MIP
* encode(network, input_constraints, output_constraints) - return MIP

In [45]:
#=
Create example problem using small_nnet and the input/output constraitns in Reverify.ipynb
=#
small_nnet = read_nnet("small_nnet.txt")
input = Constraints(zeros(1,1), zeros(1), [0.0], [0.0])
output = Constraints(zeros(2,1), zeros(1), [100.0], [0.0])
problem = Problem(small_nnet, input, output)

Problem(Network(Layer[Layer([1.0; 1.0], [1.5, 1.5]), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5]), Layer([3.0 3.0], [3.5])]), Constraints([0.0], [0.0], [0.0], [0.0]), Constraints([0.0; 0.0], [0.0], [100.0], [0.0]))

In [46]:
#=
Helper function to initialize JuMP variables associated w neurons of nnet
=#
function init_nnet_vars(model, problem)
    layers = problem.network.layers
    neurons = Array{Array{Variable}}(length(layers) + 1) # include neurons & deltas for input layer
    deltas = Array{Array{Variable}}(length(layers) + 1)
    
    # initialize variables for first layer
    n_inputs = size(layers[1].weights)[2]
    neurons[1] = @variable(model,[1:n_inputs])
    deltas[1] = @variable(model,[1:n_inputs])
    
    for (i,layer) in enumerate(layers)
        neurons[i+1] = @variable(model, [1:length(layers[i].bias)])
        deltas[i+1] = @variable(model, [1:length(layers[i].bias)], Bin)
    end
    
    return neurons, deltas
end

init_nnet_vars (generic function with 1 method)

In [47]:
#=
Helper function for adding input/output constraints to model
=#
function add_io_constraints(model, problem, neuron_vars, delta_vars)
    add_constraints(model, neuron_vars[1], problem.input)
    add_constraints(model, neuron_vars[length(problem.network.layers)], problem.output)
end

add_io_constraints (generic function with 1 method)

In [48]:
#=
Encode problem using Reverify algorithm
=#
function encode(problem)
    model = Model(solver=GLPKSolverMIP())
    neuron_vars, delta_vars = init_nnet_vars(model, problem) 
    add_io_constraints(model, problem, neuron_vars, delta_vars)
    
end

encode (generic function with 1 method)

In [49]:
encode(problem)

LoadError: [91mIncompatible sizes[39m